In [17]:
import pandas as pd
import numpy as np


df_train = pd.read_csv("train.csv")
df_train

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54268,54268,BMW,X6 xDrive50i,2017,29000,Gasoline,445.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Brown,None reported,Yes,29000
54269,54269,Audi,A4 2.0T Premium,2015,94634,E85 Flex Fuel,220.0HP 2.0L 4 Cylinder Engine Flex Fuel Capab...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,6500
54270,54270,Porsche,Cayenne S,2013,40989,Gasoline,420.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Black,At least 1 accident or damage reported,Yes,18950
54271,54271,Porsche,911 Carrera 4 GTS,2023,1518,Gasoline,4.0L H6 24V GDI DOHC,8-Speed Automatic with Auto-Shift,Beige,Brown,None reported,Yes,194965


In [18]:
df_test = pd.read_csv("test.csv")

In [19]:
mean_price_per_brand = df_train.groupby('brand')['price'].mean()

df_train['Brand_Encoded'] = df_train['brand'].map(mean_price_per_brand)
df_test['Brand_Encoded'] = df_test['brand'].map(mean_price_per_brand)

In [20]:
X_train = df_train.copy()
y_train = X_train.pop('price')

X_test = df_test.copy()

X_train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,Brand_Encoded
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,38154.063227
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,40276.029448
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,34840.403933
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,40276.029448
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,17526.060403


In [21]:
X_train.columns

Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'Brand_Encoded'],
      dtype='object')

In [22]:
#select low cardinality columns
low_cardinality_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and X_train[cname].dtype == "object"]

#select numeric columns
numeric_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

new_features = low_cardinality_cols + numeric_cols

#lets use the selected columns only 
X_train = X_train[new_features]
X_train.columns

Index(['fuel_type', 'accident', 'clean_title', 'id', 'model_year', 'milage',
       'Brand_Encoded'],
      dtype='object')

In [23]:
X_train.accident.unique()

array(['None reported', 'At least 1 accident or damage reported'],
      dtype=object)

In [24]:
from sklearn.preprocessing import OneHotEncoder

# Define a function to OHE encode
def OHE_encoder(OHE_X):
    encoder = OneHotEncoder(sparse_output=False)
    OneHotEncoded  = encoder.fit_transform(OHE_X[low_cardinality_cols])
    OHE_X = pd.DataFrame(OneHotEncoded, columns= encoder.get_feature_names_out(low_cardinality_cols))
    return OHE_X

# Encode Train and Test data
X_train_OHE = OHE_encoder(X_train)
X_train_OHE = pd.concat([X_train[numeric_cols], X_train_OHE], axis=1)
X_test_OHE = OHE_encoder(X_test)
X_test_OHE = pd.concat([X_test[numeric_cols], X_test_OHE], axis=1)


In [25]:
X_train_OHE

,id,model_year,milage,Brand_Encoded,fuel_type_Diesel,fuel_type_E85 Flex Fuel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_Plug-In Hybrid,fuel_type_not supported,fuel_type_–,accident_At least 1 accident or damage reported,accident_None reported,clean_title_Yes
0,0,2018,74349,38154.063227,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,1,2007,80000,40276.029448,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,2,2009,91491,34840.403933,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,3,2022,2437,40276.029448,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
4,4,2001,111000,17526.060403,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54268,54268,2017,29000,40276.029448,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
54269,54269,2015,94634,37091.368241,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
54270,54270,2013,40989,63742.154930,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
54271,54271,2023,1518,63742.154930,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [26]:
from sklearn.model_selection import train_test_split

train_X, valid_X, train_y, valid_y = train_test_split(X_train_OHE,y_train, test_size= 0.2)

In [27]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

model = XGBRegressor(n_estimators=1500, learning_rate=0.05, random_state=0)
model.fit(train_X, train_y, early_stopping_rounds= 5, eval_set=[(valid_X, valid_y)])

prediction_01 = model.predict(valid_X)

print("Mean Absolute Error: ", mean_absolute_error(prediction_01, valid_y))

[0]	validation_0-rmse:78688.73153
[1]	validation_0-rmse:78271.06671
[2]	validation_0-rmse:77916.52019
[3]	validation_0-rmse:77569.02469
[4]	validation_0-rmse:77281.09124
[5]	validation_0-rmse:77014.96595
[6]	validation_0-rmse:76790.88425
[7]	validation_0-rmse:76563.46792
[8]	validation_0-rmse:76337.62234
[9]	validation_0-rmse:76132.09004
[10]	validation_0-rmse:75954.64627
[11]	validation_0-rmse:75793.89763
[12]	validation_0-rmse:75659.25040
[13]	validation_0-rmse:75508.34421
[14]	validation_0-rmse:75371.48333
[15]	validation_0-rmse:75254.43506
[16]	validation_0-rmse:75107.19316
[17]	validation_0-rmse:75010.54425
[18]	validation_0-rmse:74926.60653
[19]	validation_0-rmse:74809.50780
[20]	validation_0-rmse:74691.66054
[21]	validation_0-rmse:74581.98383
[22]	validation_0-rmse:74513.79566
[23]	validation_0-rmse:74487.79738
[24]	validation_0-rmse:74435.25898
[25]	validation_0-rmse:74413.73988


c:\Users\rohan\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[26]	validation_0-rmse:74356.17729
[27]	validation_0-rmse:74297.00661
[28]	validation_0-rmse:74248.91634
[29]	validation_0-rmse:74221.52910
[30]	validation_0-rmse:74190.77338
[31]	validation_0-rmse:74165.70220
[32]	validation_0-rmse:74157.25151
[33]	validation_0-rmse:74137.58022
[34]	validation_0-rmse:74116.35428
[35]	validation_0-rmse:74092.30885
[36]	validation_0-rmse:74077.44869
[37]	validation_0-rmse:74069.52187
[38]	validation_0-rmse:74064.06021
[39]	validation_0-rmse:74061.53492
[40]	validation_0-rmse:74062.60460
[41]	validation_0-rmse:74054.66883
[42]	validation_0-rmse:74039.08687
[43]	validation_0-rmse:74024.15551
[44]	validation_0-rmse:74013.34920
[45]	validation_0-rmse:74006.72988
[46]	validation_0-rmse:73962.75337
[47]	validation_0-rmse:73944.81674
[48]	validation_0-rmse:73919.81159
[49]	validation_0-rmse:73891.82144
[50]	validation_0-rmse:73883.40265
[51]	validation_0-rmse:73876.47264
[52]	validation_0-rmse:73867.31368
[53]	validation_0-rmse:73855.02044
[54]	validation_0-rm

In [28]:
print(prediction_01)

[14277.585  12771.5205 46633.855  ... 39032.312  65749.6    12144.418 ]


In [29]:
print(y_train)

0         11000
1          8250
2         15000
3         63500
4          7850
          ...  
54268     29000
54269      6500
54270     18950
54271    194965
54272     37499
Name: price, Length: 54273, dtype: int64


In [33]:
X_test_OHE['price'] = model.predict(X_test_OHE)

In [34]:
X_test_OHE

,id,model_year,milage,Brand_Encoded,fuel_type_Diesel,fuel_type_E85 Flex Fuel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_Plug-In Hybrid,fuel_type_not supported,fuel_type_–,accident_At least 1 accident or damage reported,accident_None reported,clean_title_Yes,price
0,54273,2014,73000,47094.039119,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,29176.136719
1,54274,2015,128032,30685.441294,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,23496.740234
2,54275,2015,51983,47094.039119,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,38543.265625
3,54276,2018,29500,47714.265664,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,51506.312500
4,54277,2020,90000,40276.029448,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,43673.320312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36178,90451,2019,4500,41570.947955,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,66866.757812
36179,90452,2004,185000,44751.109855,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,14344.096680
36180,90453,2011,116000,29079.996112,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,14254.294922
36181,90454,2019,39000,27733.617992,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,43079.277344


In [35]:
X_test_OHE[['id', 'price']].to_csv("submission_1.csv", index=False) 